<a href="https://colab.research.google.com/github/akaravi/NTK.Education.Practice/blob/main/InformationSearchAndRetrieval/2-14040128/Word2Vec_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implement Word2Vec

In this assignment, you are required to implement the `Word2Vec` class from scratch using Python.

Please complete the methods inside the `Word2Vec` class:
- `__init__`: Initialize weights
- `softmax`: Impliment softmax activation function
- `train`: Loop over the data and optimize your model

You may use NumPy, but **do not** use any external word embedding libraries like Gensim or Torch for this task.

Good luck!


# Skip Gram Implementation

In [1]:
import re
import numpy as np
from collections import defaultdict
import random
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [2]:
class Vocabulary:
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.word_count = defaultdict(int)
        self.total_words = 0
        self.vocab_size = 0

    def build_vocab(self, sentences, min_count=2):
        # Count words
        for word in sentences:
            self.word_count[word] += 1

        # Create word2idx and idx2word mapping
        idx = 0
        for word, count in self.word_count.items():
            if count >= min_count:
              self.word2idx.update({word: idx})
              idx += 1

        # self.word2idx = {word: idx for idx, (word, count) in enumerate(self.word_count.items()) if count >= min_count}
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}
        self.vocab_size = len(self.word2idx)
        self.total_words = sum([count for word, count in self.word_count.items() if count >= min_count])

    def word_to_index(self, word):
        return self.word2idx.get(word, -1)

    def index_to_word(self, index):
        return self.idx2word.get(index, None)


In [3]:
def generate_training_data(vocab, sentences, window_size=2):
    training_data = []
    sentence_indices = [vocab.word_to_index(word) for word in sentences if vocab.word_to_index(word) != -1]

    for center_idx, center_word in enumerate(sentence_indices):
        context_start = max(0, center_idx - window_size)
        context_end = min(len(sentence_indices), center_idx + window_size + 1)

        for context_idx in range(context_start, context_end):
            if context_idx != center_idx:
                context_word = sentence_indices[context_idx]
                training_data.append((center_word, context_word))

    return np.array(training_data)


شروع تمرین

In [9]:
class Word2Vec:
    def __init__(self, vocab_size, embed_size=100, learning_rate=0.001):
        # سایز زا مشخص می کنم
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.learning_rate = learning_rate

        # مقداردهی اولیه ماتریس‌های وزن
        #ورودی مخفی
        self.W = np.random.uniform(-0.5, 0.5, (vocab_size, embed_size))
        #خروجی
        self.W_prime = np.random.uniform(-0.5, 0.5, (embed_size, vocab_size))

    def softmax(self, x):
        ex = np.exp(x - np.max(x))
        return ex / np.sum(ex)

    def train(self, training_data, epochs=1000):
        for epoch in range(epochs):
            loss = 0
            for c_word, context_word in training_data:
                h = self.W[c_word]
                u = np.dot(h, self.W_prime)
                y_pred = self.softmax(u)

                #encoding
                y_true = np.zeros(self.vocab_size)
                y_true[context_word] = 1

                # مجاسبه خطا
                error = y_pred - y_true

                # برروز رسانی وزن ها
                self.W_prime -= self.learning_rate * np.outer(h, error)
                self.W[c_word] -= self.learning_rate * np.dot(self.W_prime, error)

                # محاسبه looss
                loss -= np.log(y_pred[context_word])

            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Calculat Loss: {loss}')




پایان تمرین

In [6]:
# Reading the Persian stopwords from the file
from google.colab import files
# بارگذاری فایل
uploaded = files.upload()
# نام فایل بارگذاری شده
filename = list(uploaded.keys())[0]

with open(filename, 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())  # Using a set for faster lookup

# Reading the Persian stopwords from the file
#with open('persian.txt', 'r', encoding='utf-8') as f:
#    stopwords = set(f.read().splitlines())  # Using a set for faster lookup

# Sample Persian text
text = "ملکه و زن ها در کنار همسران و خانواده خود یعنی شاه و مرد ها در یک سرزمین پهناور زندگی می‌کردند شاه همیشه به مرد ها تذکر میداد که قدرت در اتحاد مرد ها و شاه نهفته است و در این قلمرو ملکه به زن ها یادآوری می‌کرد که همبستگی زن ها و ملکه مهم است و در این داستان هر مرد که نزد شاه یا زن که نزد ملکه می‌آمد از آنها حکم می‌گرفت تا به دیگران کمک کنند شاه عادل و قادر بود و ملکه خردمند و زیبا و هر مرد که از حکمت شاه یا زن که از عدالت ملکه راضی نبود نزد آنها می‌رفت تا شکایت خود را مطرح کند شاه و مرد و ملکه و زن در کنار هم بودند و هیچ کس از شاه یا ملکه نمی‌ترسید شاه همیشه به زبان میاورد که مرد ها باید به یکدیگر کمک کنند و ملکه تأکید داشتند زن ها هم باید متحد باشند"

# Tokenizing the text (you can modify the tokenizer if needed)
words = text.split()

# Removing stopwords
filtered_text = [word for word in words if word not in stopwords]

# Joining the words back into a sentence
cleaned_text = ' '.join(filtered_text)

print(cleaned_text)


Saving persian.txt to persian.txt
ملکه زن همسران خانواده شاه مرد سرزمین پهناور زندگی می‌کردند شاه مرد تذکر میداد قدرت اتحاد مرد شاه نهفته قلمرو ملکه زن یادآوری می‌کرد همبستگی زن ملکه مهم داستان مرد شاه زن ملکه می‌آمد حکم می‌گرفت کمک شاه عادل قادر ملکه خردمند زیبا مرد حکمت شاه زن عدالت ملکه راضی می‌رفت شکایت مطرح شاه مرد ملکه زن شاه ملکه نمی‌ترسید شاه زبان میاورد مرد کمک ملکه تأکید زن متحد


In [7]:
# wiki_dump_path = 'enwiki-latest-pages-articles.xml.bz2'  # Path to your Wikipedia dump file

# # Load and preprocess the dataset
# sentences = list(load_wiki_data(wiki_dump_path))

# Build the vocabulary
vocab = Vocabulary()
vocab.build_vocab(cleaned_text.split(' '))

# Generate training data
training_data = generate_training_data(vocab, cleaned_text.split(' '))

# Initialize and train Word2Vec model
word2vec_model = Word2Vec(vocab.vocab_size)
word2vec_model.train(training_data, epochs=1000)


Epoch 0, Loss: 245.10641459946234
Epoch 100, Loss: 181.74829719507156
Epoch 200, Loss: 181.16495769308588
Epoch 300, Loss: 180.98387727437643
Epoch 400, Loss: 180.8973966357685
Epoch 500, Loss: 180.8470350481156
Epoch 600, Loss: 180.81433746041418
Epoch 700, Loss: 180.79156394672066
Epoch 800, Loss: 180.77489919338907
Epoch 900, Loss: 180.76224974387054


Defines a function to retrieve the word embedding for a given word from a Word2Vec model. If the word exists in the vocabulary, its corresponding vector is returned; otherwise, None is returned.

In [8]:
def get_word_embedding(word, vocab, model):
    word_idx = vocab.word_to_index(word)
    if word_idx != -1:
        return model.W[word_idx]
    else:
        return None

embedding = get_word_embedding("مرد", vocab, word2vec_model)
print(embedding)


[-0.16789147 -0.5261984  -0.36680291 -0.32044157 -0.3287403  -0.53621258
 -0.1938785  -0.05296368  0.0776731   0.02722436 -0.44692452  0.35983948
 -0.39242053 -0.38250574  0.33880482 -0.12671234 -0.47161344  0.33217644
  0.39797687 -0.37405056 -0.14654314 -0.10143383  0.17590998 -0.06549403
  0.33954333 -0.04125304  0.05853389 -0.37565101  0.04373925 -0.38967107
 -0.33048455 -0.25614055 -0.27436118  0.00433121 -0.34681916 -0.01036167
 -0.00373732  0.07333587 -0.3698033   0.28749311 -0.01163859  0.327175
 -0.27677611  0.28118788  0.42189819 -0.12623055 -0.04848502  0.02863034
 -0.00240019  0.47868774 -0.3371656   0.0255005   0.27518353  0.16212707
  0.17439922  0.27733184  0.40814703 -0.33843478 -0.47383956  0.09222321
 -0.23422742  0.20034953  0.18606345  0.34380499  0.35347779  0.20304626
 -0.4663782   0.23560661  0.04446361 -0.34564012  0.44997474  0.09978872
  0.22091428 -0.45096951  0.29421306  0.28610365 -0.0728715   0.05970688
  0.21843747  0.39441081  0.12348063  0.06592447  0.2